# Libraries und arbeiten mit APIs

#### 0. Importiere die Library requests.

In [19]:
import requests
import pandas as pd
import datetime

#### 1. Suche 3 Beispiele von APIs. Von solchen, die mit und ohne Schlüssel, verwendet werden können und liste sie auf. (Ihr müsst sie nicht gleich auslesen. Sondern einfach mal hier die Dokumentation aufführen).

In [20]:
# https://www.parlament.ch/de/services/open-data-webservices Die API des Schweizer Parlaments
# https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/forschung/api.html API-Zugänge BfS
# https://data.sbb.ch/explore/?sort=modified Die API der SBB
# https://www.post.ch/de/geschaeftsloesungen/digital-commerce/digital-commerce-api/personalisierte-api-integration #Die API der Post
# https://rapidapi.com/products/enterprise-hub/ Unternehmen
# https://github.com/r-spacex/SpaceX-API SpaceX
# https://api.nasa.gov/


# https://developers.google.com/apis-explorer Googles Super Angebot
# https://developer.twitter.com/en/docs Twitter
# ...



- [Schweizer Parlament](https://www.parlament.ch/de/services/open-data-webservices)
- [BfS](https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/forschung/api.html)
- [SBB](https://data.sbb.ch/explore/?sort=modified)
- [Post](https://www.post.ch/de/geschaeftsloesungen/digital-commerce/digital-commerce-api/personalisierte-api-integration)
- [Google](https://developers.google.com/apis-explorer)
- [Twitter](https://developer.twitter.com/en/docs)

#### 2. Lese eine dieser APIs mit requests ein und rufe die Ergebnisse auf.

In [2]:
r = requests.get("http://ws-old.parlament.ch/sessions?format=json")

In [10]:
r.text

'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head id="ctl00_Head1"><title>\r\n\t\r\n    Sessions\r\n - Web Services des Schweizer Parlaments \r\n</title><link href="Content/Site.css" rel="stylesheet" type="text/css" /><link href="Content/menu.css" rel="stylesheet" type="text/css" />\r\n    <script type="text/javascript" src="../../Content/jquery.min.js"></script>\r\n    <script type="text/javascript" src="../../Content/menu.js"></script>\r\n\r\n    <meta name="viewport" content="width=device-width,initial-scale=1" /></head>\r\n\r\n<body>\r\n    <div class="page">\r\n\r\n        <div id="header">\r\n            <div id="title">\r\n                <h1>Web Services des Schweizer Parlaments</h1>\r\n            </div>\r\n            \r\n            <div id="logindisplay">\r\n                \r\n           <span class="lang-active" title="Deutsch">DE</span>\r\n        

#### 3. Zurück zum [Erdbeben API](https://earthquake.usgs.gov). Wie viele Erdbeben wurden im Januar 2020 verzeichnet?

In [11]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2020-01-01&endtime=2020-01-31')

In [12]:
r.json()

16249

#### 4. Und wie viele Erdbeben im Juli?

In [13]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2020-07-01&endtime=2020-07-31')

In [14]:
r.json()

17343

#### 5. In welchem Monat des aktuellen Jahres wurden am meisten Erdbeben verzeichnet?

In [127]:
dates_start= pd.date_range('2021-01-01','2021-11-01' , freq='1M')-pd.offsets.MonthBegin(1)

In [128]:
dates_start

DatetimeIndex(['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01'],
              dtype='datetime64[ns]', freq=None)

In [129]:
dates_end= pd.date_range('2021-02-01','2021-11-01' , freq='1M')-pd.offsets.MonthEnd(1)

In [130]:
dates_end

DatetimeIndex(['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
               '2021-09-30'],
              dtype='datetime64[ns]', freq=None)

In [131]:
base = 'https://earthquake.usgs.gov/fdsnws/event/1/count?'
start = 'starttime='
end = '&endtime='

In [132]:
liste = []
for s, e in zip(dates_start, dates_end):
    s = str(s)
    e = str(e)
    r = requests.get(base+start+s+end+e)
    result = r.text
    
    mini_dict = {s:result}
    liste.append(mini_dict)

In [133]:
pd.DataFrame(liste).melt().dropna()

,variable,value
0,2021-01-01 00:00:00,13863
10,2021-02-01 00:00:00,11438
20,2021-03-01 00:00:00,13117
30,2021-04-01 00:00:00,12421
40,2021-05-01 00:00:00,11171
50,2021-06-01 00:00:00,13285
60,2021-07-01 00:00:00,15719
70,2021-08-01 00:00:00,15680
80,2021-09-01 00:00:00,4466


## Die 10 stärksten Erdbeben im 2021

In [149]:
url1 = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'
urlzeit = '&starttime=2010-01-01&endtime=2021-09-13'
minmax = "&minmagnitude=6.5"

In [150]:
response = requests.get(url1 + urlzeit+minmax)

In [151]:
dct = response.json()

In [152]:
len(dct['features'])

554

In [153]:
earthquakes = []

for elem in dct['features']:
    
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    tm = elem['properties']['time']
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

In [154]:
pd.DataFrame(earthquakes).sort_values(by='Mag', ascending=False).head(10)

,Type,Mag,Place,Time
478,earthquake,9.1,"2011 Great Tohoku Earthquake, Japan",1299822384120
545,earthquake,8.8,"36 km WNW of Quirihue, Chile",1267252451530
422,earthquake,8.6,off the west coast of northern Sumatra,1334133516720
250,earthquake,8.3,"48 km W of Illapel, Chile",1442444072860
369,earthquake,8.3,Sea of Okhotsk,1369374288980
421,earthquake,8.2,off the west coast of northern Sumatra,1334140990850
126,earthquake,8.2,"267 km E of Levuka, Fiji",1534637980670
11,earthquake,8.2,"99 km SE of Perryville, Alaska",1627539349188
330,earthquake,8.2,"93 km NW of Iquique, Chile",1396396007260
157,earthquake,8.2,"near the coast of Chiapas, Mexico",1504846159180


In [155]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-01-01&endtime=2021-09-12'
orderby = '&orderby=magnitude'
limit = '&limit=10'
r = requests.get(url+orderby+limit)
r.json()['features']

[{'type': 'Feature',
  'properties': {'mag': 8.2,
   'place': '99 km SE of Perryville, Alaska',
   'time': 1627539349188,
   'updated': 1631515681868,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak0219neiszm',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak0219neiszm&format=geojson',
   'felt': 268,
   'cdi': 8.1,
   'mmi': 7.355,
   'alert': 'yellow',
   'status': 'reviewed',
   'tsunami': 1,
   'sig': 1252,
   'net': 'ak',
   'code': '0219neiszm',
   'ids': ',ak0219neiszm,us6000f02w,at00qwzteb,pt21210001,usauto6000f02w,',
   'sources': ',ak,us,at,pt,usauto,',
   'types': ',dyfi,finite-fault,general-text,ground-failure,impact-link,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,oaf,origin,phase-data,shakemap,trump-losspager,trump-shakemap,',
   'nst': None,
   'dmin': None,
   'rms': 0.95,
   'gap': None,
   'magType': 'mww',
   'type': 'earthquake',
   'title': 'M 8.2 - 99 km SE of Perryville, Alaska